In [3]:
# Import required packages
from huggingface_hub import snapshot_download
import os

# Download ProDiff weights from Hugging Face
print("Downloading ProDiff weights...")
downloaded_path = snapshot_download(repo_id="Rongjiehuang/ProDiff")

# Create checkpoints directory if it doesn't exist
os.makedirs("checkpoints", exist_ok=True)

# Move downloaded checkpoints to the proper location
if os.path.exists(os.path.join(downloaded_path, "checkpoints")):
    import shutil
    print("Moving checkpoints to project directory...")
    shutil.copytree(
        os.path.join(downloaded_path, "checkpoints"),
        "checkpoints",
        dirs_exist_ok=True
    )
    print("ProDiff weights downloaded and set up successfully!")
else:
    print("Warning: Checkpoints folder not found in downloaded content")

# The models will now be available in:
# - checkpoints/ProDiff Teacher/ (for teacher model)
# - checkpoints/ProDiff/ (for student model)

# Check if ProDiff directory already exists
if os.path.exists('ProDiff'):
    print("ProDiff repository already exists")
else:
    !git clone https://github.com/Rongjiehuang/ProDiff.git

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Moving checkpoints to project directory...
ProDiff weights downloaded and set up successfully!
Cloning into 'ProDiff'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 181 (delta 20), reused 10 (delta 10), pack-reused 120 (from 1)
Receiving objects: 100% (181/181), 184.89 KiB | 7.40 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [7]:
# Import necessary modules
import torch
from ProDiff.modules.ProDiff.config.config import Config
from ProDiff.modules.ProDiff.model.prodiff import ProDiff
from ProDiff.modules.ProDiff.text import text_to_sequence

# Load config
config = Config("modules/ProDiff/config/prodiff.yaml")

# Initialize model
model = ProDiff(config)

# Load checkpoint
checkpoint = torch.load("checkpoints/ProDiff/model_ckpt_steps_100000.ckpt", map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])

# Set model to evaluation mode
model.eval()

# Prepare input text
text = "hello world"
sequence = text_to_sequence(text)
sequence = torch.LongTensor(sequence).unsqueeze(0)

# Generate audio
with torch.no_grad():
    audio = model.inference(sequence)

# Convert to numpy array and save
import numpy as np
import soundfile as sf

audio = audio.squeeze().cpu().numpy()
sf.write('output.wav', audio, config.data.sampling_rate)

print(f"Generated audio saved as 'output.wav'")


ModuleNotFoundError: No module named 'ProDiff.modules.ProDiff.config.config'